In [1]:
import warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)

import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly

import datetime as dt
import calendar

import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows

from Position_Report_Calcs import Position_Reporting

In [2]:
plotly.offline.init_notebook_mode(connected=True)

In [3]:
## Read in the position data
mkts = ['ACCU','NZU','EUA','OTHER']

positions = dict()
for m in mkts:
    positions[m] = pd.read_excel('Positions.xlsx', sheet_name=m)
    positions[m]['Expiry'] = pd.to_datetime(positions[m].Expiry).dt.date

premiums = pd.read_excel('Positions.xlsx', sheet_name='Index') 

In [4]:
## Define the dates we want to use for reporting
current_date = dt.datetime.today().date()
current_year = str(dt.datetime.today().date().year)

_, last_day = calendar.monthrange(current_date.year, current_date.month)
_, last_day_next_month = calendar.monthrange(current_date.year, current_date.month+1)

end_of_month = dt.date(current_date.year, current_date.month, last_day)
end_of_next_month = dt.date(current_date.year, current_date.month + 1, last_day_next_month)

one_month = current_date + dt.timedelta(days=1*30)

three_months = current_date + dt.timedelta(days=3*30)

six_months = current_date + dt.timedelta(days=6*30)

eoy = '10-12-{}'.format(current_year)
end_of_year = dt.datetime.strptime(eoy, '%d-%m-%Y').date()

one_year = current_date + dt.timedelta(days=365)

end_of_may = dt.datetime.strptime('31-05-2024', '%d-%m-%Y').date()

june_expiry = dt.datetime.strptime('15-06-2024', '%d-%m-%Y').date()

dates_names = ['today','EoM','End of Next Month','3 months','6 months','EoY','OneYear','End of May', 'Mid June']
dates_values = [current_date, end_of_month, end_of_next_month, three_months, six_months, end_of_year, one_year,end_of_may,june_expiry]

In [5]:
dates = dict()
for i in list(range(0,len(dates_names))):
    dates[dates_names[i]] = dates_values[i]

In [6]:
current_spot = positions['OTHER']
current_spot = current_spot[current_spot.Type=='Spot'].reset_index(drop=True)
current_spot = current_spot['Price'][0]

# OTHER Position Report
All PnL figures are in AUD

In [7]:
other = dict()
workbook = openpyxl.Workbook()

for i,d in enumerate(dates_values):
    other[d] = Position_Reporting(positions, 'OTHER', d).combine_frame()
    
    sheet_name = dates_names[i]
    sheet = workbook.create_sheet(title=sheet_name)
    
    for row in dataframe_to_rows(other[d][0], index=False, header=True):
        sheet.append(row)
    
workbook.save('Other PnL Output.xlsx')    

In [8]:
# data type; 0=PnL, 1=Delta, 2=Option Theta, 3=Option Vega
def generate_summary_plots(mkt_data, data_type):
    if data_type==0:
        colname='Total_Pnl'
    elif data_type==1:
        colname='Total_Delta'
    elif data_type==2:
        colname='Option_Theta'
    elif data_type==3:
        colname='Option_Vega'
    fig = go.Figure()
    for i in list(range(0, len(dates_names))):
        date_value = dates_values[i]
        date_name = dates_names[i]
        fig.add_trace(go.Scatter(x=mkt_data[date_value][data_type].Price, y=mkt_data[date_value][data_type][colname], name=date_name))
    return fig

In [9]:
#fig_d = generate_summary_plots(other, 1)
#fig_p = generate_summary_plots(other, 0)
#fig_t = generate_summary_plots(other,2)
#fig_v = generate_summary_plots(other,3)

In [10]:
#fig = make_subplots(rows=1, cols=2)
#for i in range(0,len(fig_d.data)):
#    fig.add_trace(fig_p.data[i], row=1, col=1)
#    fig.add_trace(fig_d.data[i], row=1, col=2)
#
#fig.update_layout(title={'text': 'OTHER Position (PnL and Delta)','x': 0.5,'xanchor': 'center'})
#
#fig.update_xaxes(title_text="Price", row=1, col=1)
#fig.update_yaxes(title_text="PnL", row=1, col=1)
#fig.update_xaxes(title_text="Price", row=1, col=2)
#fig.update_yaxes(title_text="Delta", row=1, col=2)
#
#fig.add_vline(x=current_spot, line_width=1, line_dash="dash", line_color="green")
#
#fig.show()

In [11]:
#fig = make_subplots(rows=1, cols=2)
#for i in range(0,len(fig_d.data)):
#    fig.add_trace(fig_t.data[i], row=1, col=1)
#    fig.add_trace(fig_v.data[i], row=1, col=2)
#
#fig.update_layout(title={'text': 'OTHER Position (Theta and Vega)','x': 0.5,'xanchor': 'center'})
#
#fig.update_xaxes(title_text="Price", row=1, col=1)
#fig.update_yaxes(title_text="Theta", row=1, col=1)
#fig.update_xaxes(title_text="Price", row=1, col=2)
#fig.update_yaxes(title_text="Vega", row=1, col=2)
#fig.show()

In [12]:
## USE THIS STYLE OF GRAPHING FOR TODAY DATE ENTRY ##
#position_functions = Position_Reporting(positions, 'OTHER', current_date)
#pnl, delta, theta, vega, value = position_functions.combine_frame()
#
#fig_d = go.Figure()
#
#fig_d.add_trace(go.Bar(x=delta.Price, y=delta.Spot, name='Spot', base=0))
#fig_d.add_trace(go.Bar(x=delta.Price, y=delta.Fwds, name='Fwds', base=0))
#fig_d.add_trace(go.Bar(x=delta.Price, y=delta.Options, name='Options', base=0))
#fig_d.add_trace(go.Scatter(x=delta.Price, y=delta.Total_Delta, mode='lines', name='Total_Delta'))
#
#
##~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#fig_p = go.Figure()
#
#fig_p.add_trace(go.Bar(x=pnl.Price, y=pnl.Spot, name='Spot', base=0))
#fig_p.add_trace(go.Bar(x=pnl.Price, y=pnl.Fwds, name='Fwds', base=0))
#fig_p.add_trace(go.Bar(x=pnl.Price, y=pnl.Options, name='Options', base=0))
#fig_p.add_trace(go.Scatter(x=pnl.Price, y=pnl.Total_Pnl, mode='lines', name='Total_PnL'))
#
#
##~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#fig = make_subplots(rows=1, cols=2)
#for i in range(0,len(fig_d.data)):
#    fig.add_trace(fig_d.data[i], row=1, col=2)
#    fig.add_trace(fig_p.data[i], row=1, col=1)
#    
#fig.update_layout(title={'text': 'Todays OTHER Pos (PnL and Delta)','x': 0.5,'xanchor': 'center'}, barmode='stack')
#
#fig.update_xaxes(title_text="Price", row=1, col=1)
#fig.update_yaxes(title_text="Pnl", row=1, col=1)
#fig.update_xaxes(title_text="Price", row=1, col=2)
#fig.update_yaxes(title_text="Delta", row=1, col=2)    
#fig.show()  

### Positions

In [13]:
positions['OTHER']

,Name,Type,Subtype,Strike,Vol,Expiry,Qty,Price,Rate,FX
0,EUA Dec24,Spot,NaN,NaN,NaN,NaT,0,62.010000,0.032325,1.62845
1,EUA Dec24 75C,Option,Call,75.0,0.383876,2024-12-16,25000,0.689892,0.032325,1.62845
2,EUA Dec24 80C,Option,Call,80.0,0.395457,2024-12-16,50000,0.349333,0.032325,1.62845
3,EUA Dec24 85C,Option,Call,85.0,0.408208,2024-12-16,100000,0.181692,0.032325,1.62845


In [14]:
# Options for timing are: [current_date, end_of_month, one_month, three_months, six_months, end_of_year, end_of_feb, march_expiry]

def pnl_delta_plot(timing):
    position_functions = Position_Reporting(positions, 'OTHER', timing)
    p,d,t,v, value = position_functions.options()
    fig_p = go.Figure()
    for i in list(p)[1:-1]:
        fig_p.add_trace(go.Scatter(x=p.Price, y=p[i], name=i))

    fig_p.add_trace(go.Scatter(x=p.Price, y=p.Option_Pnl, mode='lines', name='Total_PnL'))

    #fig_p.update_layout(title='Option PnL Today', xaxis_title='Price', yaxis_title='pnl', barmode='stack')
    #fig_p.show()

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    fig_d = go.Figure()
    for i in list(d)[1:-1]:
        fig_d.add_trace(go.Scatter(x=d.Price, y=d[i], name=i))

    fig_d.add_trace(go.Scatter(x=d.Price, y=d.Option_Delta, mode='lines', name='Total_Delta'))

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    fig = make_subplots(rows=1, cols=2)
    for i in range(0,len(fig_p.data)):
        fig.add_trace(fig_p.data[i], row=1, col=1)
        fig.add_trace(fig_d.data[i], row=1, col=2)

    fig.update_layout(title={'text': 'Todays Option Pos (PnL and Delta)','x': 0.5,'xanchor': 'center'}, barmode='stack')

    fig.update_xaxes(title_text="Price", row=1, col=1)
    fig.update_yaxes(title_text="Pnl", row=1, col=1)
    fig.update_xaxes(title_text="Price", row=1, col=2)
    fig.update_yaxes(title_text="Delta", row=1, col=2)    
    fig.show()  

## Current Date

In [15]:
pnl_delta_plot(current_date)

## End of Month

In [16]:
pnl_delta_plot(end_of_month)

## One Month

In [17]:
pnl_delta_plot(one_month)

## End of Next Month

In [18]:
pnl_delta_plot(end_of_next_month)

## Mid June

In [19]:
pnl_delta_plot(june_expiry)

## End of Year

In [20]:
pnl_delta_plot(end_of_year)

In [21]:
### THIS IS FOR PLOTTING THETA AND VEGA

#fig_t = go.Figure()
#for i in list(t)[1:-1]:
#    fig_t.add_trace(go.Bar(x=t.Price, y=t[i], name=i, base=0))
#
#fig_t.add_trace(go.Scatter(x=t.Price, y=t.Option_Theta, mode='lines', name='Total_Theta'))
#
#
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#fig_v = go.Figure()
#for i in list(v)[1:-1]:
#    fig_v.add_trace(go.Bar(x=v.Price, y=v[i], name=i, base=0))
#
#fig_v.add_trace(go.Scatter(x=v.Price, y=v.Option_Vega, mode='lines', name='Total_Vega'))
#
##~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#fig = make_subplots(rows=1, cols=2)
#for i in range(0,len(fig_p.data)):
#    fig.add_trace(fig_t.data[i], row=1, col=1)
#    fig.add_trace(fig_v.data[i], row=1, col=2)
#   
#fig.update_layout(title={'text': 'Todays Option Pos (Theta and Vega)','x': 0.5,'xanchor': 'center'}, barmode='stack')
#
#fig.update_xaxes(title_text="Price", row=1, col=1)
#fig.update_yaxes(title_text="Theta", row=1, col=1)
#fig.update_xaxes(title_text="Price", row=1, col=2)
#fig.update_yaxes(title_text="Vega", row=1, col=2)    
#fig.show()  